In [ ]:
import Brunel
import datetime
import random

In [ ]:
import pandas as pd

In [ ]:
nodes = pd.read_csv("input/nodes.csv")

In [ ]:
edges = pd.read_csv("input/edges.csv")

In [ ]:
nodes

In [ ]:
edges

Adding some random dates so that I can test the temporal controls

In [ ]:
days = datetime.timedelta(days=1)
weeks = datetime.timedelta(weeks=1)
months = 30*days
years = datetime.timedelta(days=365)

In [ ]:
def random_date(start=datetime.date(year=1800, month=1, day=1), end=datetime.date(year=1890, month=12, day=31),
                within=None):
    if within:
        start = within[0]
        end = within[1]

    start = datetime.datetime.combine(start, datetime.time()).timestamp()
    end = datetime.datetime.combine(end, datetime.time()).timestamp()
    result = start + random.random() * (end - start)
    return datetime.datetime.fromtimestamp(result).date()

In [ ]:
def random_duration(start=datetime.date(year=1800, month=1, day=1), 
                    end=datetime.date(year=1890, month=12, day=31),
                    within=None,
                    minimum=20*years, maximum=80*years,
                    breach_maximum=False):
    if within:
        start = within[0]
        end = within[1]
    mindur = minimum.total_seconds()
    maxdur = maximum.total_seconds()
    start = random_date(start=start, end=end)

    if not breach_maximum:
        lifedur = (end - start).total_seconds()
        if maxdur > lifedur:
            maxdur = lifedur
        if mindur > maxdur:
            mindur = 0.5*maxdur

    dur = mindur + random.random() * (maxdur-mindur)

    return (start, start + datetime.timedelta(seconds=dur))

In [ ]:
def random_lifetime(start=datetime.date(year=1800, month=1, day=1),
                    end=datetime.date(year=1890, month=12, day=31),
                    maximum_age=80*years, all_adults=True):
    if all_adults:
        minimum = 18*years
    else:
        minimum = 1*day
        
    return random_duration(start=start, end=end, minimum=minimum, maximum=maximum_age, breach_maximum=True)

In [ ]:
def adult(lifetime):
    start = lifetime[0]
    end = lifetime[1]
    start = start + 18*years
    if start > end:
        raise ValueError("Not an adult %s => %s" % (start.isoformat(), end.isoformat()))
    return (start, end)

In [ ]:
lifetime = random_lifetime()
print(lifetime)
print(adult(lifetime))
print(random_duration(within=adult(lifetime), minimum=6*months, maximum=5*years))
print(f"Lived {(lifetime[1]-lifetime[0]).total_seconds() / (3600*24*365)} years")

In [ ]:
position = random_duration(within=lifetime, minimum=6*months, maximum=5*years)
position

In [ ]:
def add_random_dates_to_node(node):
    print(item.__dict__)
    return item

In [ ]:
def add_random_dates_to_message(message):
    print(message)

In [ ]:
social = Brunel.Social.load_from_csv("input/nodes.csv", "input/edges.csv",
                                     modifiers={"person": add_random_date,
                                                "business": add_random_dates_to_node,
                                                "message": add_random_dates_to_message})

In [ ]:
p = social.state["messages"]